<a href="https://colab.research.google.com/github/hideousmaiden/semantics_handouts/blob/main/%D0%B0%D0%B2%D1%82%D0%BE%D1%80%D0%B0%D1%81%D1%81%D1%8B%D0%BB%D0%BA%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install secure-smtplib==0.1.1

In [ ]:
import os
import random
import pandas as pd
from google.colab import files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#СЮДА ВСТАВИТЬ ПУТЬ К ПАПКЕ КУРСА
#СДЕЛАТЬ В ПАПКЕ КУРСА ПАПКУ "rassylka"
os.chdir('/content/drive/MyDrive/формсем_2025_преподская')

In [ ]:
#ПЕРЕНЕСТИ ПАПКИ С ТЕМАМИ В rassylka
problemdir = '/content/drive/MyDrive/формсем_2025_преподская/rassylka'
def problem_chooser():
    problems = []
    for subdir, dirs, files in os.walk(problemdir):
       for dir in dirs:
           thing = random.choice(os.listdir(os.path.join(problemdir, dir)))
           problems.append((os.path.join(problemdir, dir, thing), thing))
    return problems

In [ ]:
print(problem_chooser()[1])

('/content/drive/MyDrive/формсем_2025_преподская/rassylka/4/4А.pdf', '4А.pdf')


In [ ]:
# ВСТАВИТЬ ПОЧТУ
sender_email = "sidorckina.darya@gmail.com"
#ЭТО НЕ ТРОГАТЬ
# The port number identifies specific processes when an internet or network message is forwarded to a server.
port = 0 # If port is zero, the standard SMTP-over-SSL port (465) is used.
smtp_server = "smtp.gmail.com" # Set SMTP server provider or SMTP setting

# ВСТАВИТЬ СГЕНЕРИРОВАННЫЙ ПАРОЛЬ
# Please read and follow in the link: https://support.google.com/accounts/answer/185833?visit_id=638256356071361680-2723346452&p=InvalidSecondFactor&rd=1
password = "" # Change your password

In [ ]:
#текст письма
subject = "Задачи по формсему"
message = """
Дорогая/дорогой {},

это письмо с вашими задачами по формальной семантике. Здесь задачи на первые 4 темы. Напоминаю, что дедлайн для их письменной сдачи - 8 октября.

--------------------------------------------------------------------------
Хороших вам грядущих выходных,
Даша.
"""

In [ ]:
#подкл к таблице
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
#название таблички со списком студентов и их почт из папки курса
worksheet = gc.open('формсем_2025_домашки_преподская').get_worksheet(1) #номер листа, начинаются с нуля

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
headd = rows[0]
rows = rows[1:]
print(rows)

df = pd.DataFrame.from_records(rows, columns=headd)
df.head()

[['Фуркин Николай', 'nafurkin@edu.hse.ru'], ['Марьин Даниил', 'dsmarin@edu.hse.ru'], ['Полина Ратанова', 'pvratanova@edu.hse.ru'], ['Петухова Виктория', 'vvpetukhova_2@edu.hse.ru'], ['Шнайдштейн Александр', 'avshnaidshtein@edu.hse.ru'], ['Седунова Софья', 'svsedunova@edu.hse.ru']]


,фамилия имя,почта
0,Фуркин Николай,nafurkin@edu.hse.ru
1,Марьин Даниил,dsmarin@edu.hse.ru
2,Полина Ратанова,pvratanova@edu.hse.ru
3,Петухова Виктория,vvpetukhova_2@edu.hse.ru
4,Шнайдштейн Александр,avshnaidshtein@edu.hse.ru


In [ ]:
import smtplib, ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage
from email.mime.application import MIMEApplication

In [ ]:
def email_sender(row):
    stud_name = row['фамилия имя']
    stud_email = row['почта']
    problems = problem_chooser()

    # Create a MIMEMultipart object to handle multiple parts of the message
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = stud_email
    msg['Subject'] = subject
    out = []

    #записываем в табличку, какой вариант дали

    # Attach the plain text message to the email
    msg.attach(MIMEText(message.format(stud_name), 'plain'))
    count = 1
    for problem in problems:
        row[str(count)] = problem[1].split('.')[0]
        count += 1
        # Attach the PDF file to the email
        with open(problem[0], 'rb') as PDF_file:
            PDF_attachment = MIMEApplication(PDF_file.read(), name=problem[1])
            msg.attach(PDF_attachment)
    # Send the email
    context = ssl.create_default_context()
    with smtplib.SMTP_SSL(smtp_server, port, context=context) as server:
        server.login(sender_email, password)
        server.sendmail(sender_email, stud_email, msg.as_string())
    print("Successfully sent email")
    return row

In [ ]:
#email_sender('Шаша', 'sidorckina.darya@yandex.ru')

In [ ]:
#тест-драйв на коллегах
#data = {'фамилия имя': ['Шаша','Полупавел', 'АВ'],
#        'почта': ['sidorckina.darya@yandex.ru', 'pastafev760@gmail.com', 'sasha.podobryaev@gmail.com']}
#data = {'фамилия имя': ['Шаша',],
#       'почта': ['sidorckina.darya@yandex.ru']}
# Create DataFrame
#df = pd.DataFrame(data)

#print(df)

In [ ]:
#РАССЫЛКА!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!1

df = df.apply(email_sender, axis=1)
print(df)

Successfully sent email
Successfully sent email
Successfully sent email
Successfully sent email
Successfully sent email
Successfully sent email
            фамилия имя                      почта   1   2   3   4
0        Фуркин Николай        nafurkin@edu.hse.ru  2А  4Б  1А  3А
1         Марьин Даниил         dsmarin@edu.hse.ru  2А  4В  1Б  3А
2       Полина Ратанова      pvratanova@edu.hse.ru  2А  4А  1А  3В
3     Петухова Виктория   vvpetukhova_2@edu.hse.ru  2Б  4Г  1Г  3В
4  Шнайдштейн Александр  avshnaidshtein@edu.hse.ru  2В  4Б  1В  3В
5        Седунова Софья      svsedunova@edu.hse.ru  2А  4В  1В  3А


In [ ]:
#ЗАГРУЗИТЬ ПОТОМ СПИСОК!!!!!!!!!!!!!!!1
df.to_csv('df.csv')
files.download('df.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>